In [ ]:
!pip install agentpy==0.0.7
!pip install get_variable_name==0.0.2
!pip install transformers==4.4.2
!pip install vaderSentiment==3.3.2
!pip install torch==1.8.1

In [15]:
#https://hub.gke2.mybinder.org/user/joelforamitti-agentpy_workshop-ni6y1xd2/notebooks/agentpy_workshop.ipynb
#Load in packages

import agentpy as ap
import numpy as np 
from numpy import mean
import matplotlib.pyplot as plt
import pandas as pd

import random
import get_variable_name
import itertools

from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer
import torch

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
SIA = SentimentIntensityAnalyzer() #Create sentiment analyser object

In [3]:
#Import a chatbot
my_bot1 = BlenderbotForConditionalGeneration.from_pretrained('facebook/blenderbot-400M-distill')

#import tokenizer object needed for chatbot operation
tokenizer = BlenderbotTokenizer.from_pretrained('facebook/blenderbot-400M-distill')



In [79]:
class ChatbotAgent(ap.Agent):
    """ a chatbot agent"""
    def setup(self):
        self.chatbot = self.p.chatbot
        self.preferedSentiment = random.uniform(-1, 1)


        #Set up environment knowledge of the chatbot
        self.environmentKnowledge = [0]
        self.weighted_average = 0 #np.average(self.environmentKnowledge[-10:], weights=list(range(1,len(self.environmentKnowledge[-10:])+1)))

        #Set up variables to be recorded
        self.agent1 = []
        self.agent2 = []
        self.conversation_starter = []
        self.my_id = []
        self.partner_id = []
        self.sentiment1 = []
        self.sentiment2 = []
        self.preferedSentiment1 = []
        self.preferedSentiment2 = []
        self.sentimentRange = []
        self.t = 0

    def chatbot_conversation(self):
        #if (self.t == 600):
        #    partnerID = random.choice([x for x in range(0, self.p.agents) if x != (self.id-1)]) #Choose partner - can choose any chatbot but itself
        #    partner = self.model.agents[partnerID]

        
        dist = []
        #Model distance between opinion of agent i and all other agents
 

        for i in range(self.p.agents):
            dist.append(self.weighted_average-self.model.agents.weighted_average[i])

            
        #Calculate probability for each
        res =  [(abs(ele)+0.01)**(-self.p.beta)/((abs(sum(dist)))+0.01)**(-self.p.beta) for ele in dist]


        #Remove our own chatbot
        del res[self.id-1]
        if sum(res) == 0: #Avoid dividing by zerro
            res = [1]*(self.p.agents-1)
        

        partnerIDX = random.choices(
            [x for x in range(0, self.p.agents) if x != (self.id-1)], 
            k = 1, #Pick one agent from the list
            weights = res
            )[0]

        partner = self.model.agents[partnerIDX]


        #Set up temporary variables
        tempAgent1 = []
        tempAgent2 = []
        tsentiment1 = []
        tsentiment2 = []
        tsentimentRange = []
        entireConv = ""

        #Pick random conversation starter from input list and tokenize
        conversation_starter = random.choice(self.p.conversation_starter)
        encoded_conv_starter = self.p.tokenizer(conversation_starter, return_tensors='pt').input_ids
        
        j = 0
        while j < self.p.conversation_length: #Conversation loop
            
            responses_decoded1 = []
            responses_decoded2 = []
            tempSentiment1 = []
            tempSentiment2 = []
            
            
            #Turn for first chatbot
            if j == 0:
                responses = self.chatbot.generate(
                    encoded_conv_starter,
                    max_length=30, # maximum number of tokens
                    num_beams=10, # options kept active at the same time
                    num_return_sequences=10, # how many of the candidates to return, has to be <= num_beams
                    no_repeat_ngram_size=2, # this penalized probability of sequences with internal repetition
                    early_stopping=True,
                    #top_k = 50, 
                    #top_p = 0.6,
                    temperature= 2.0,
                    do_sample=True)

                for i, response in enumerate(responses):
                    responses_decoded1.append(self.p.tokenizer.decode(response, skip_special_tokens=True))
                    tempSentiment1.append(self.p.SIA.polarity_scores(responses_decoded1[i])['compound'])

                #get ID of sentence most similar to prefered sentiment
                ID = min(range(len(tempSentiment1)), key=lambda i: abs(tempSentiment1[i]-self.preferedSentiment))    

                tempAgent1.append(responses_decoded1[ID])
                tsentiment1.append(tempSentiment1[ID])
                tsentimentRange.append([min(tempSentiment1), max(tempSentiment1)])
                entireConv = entireConv + responses_decoded1[ID]


            else:
                
                if j > 1:
                    entireConv = entireConv.split("    ", 1)[1]
                encoded_conv = self.p.tokenizer(entireConv, return_tensors='pt').input_ids

                responses = self.chatbot.generate(
                    encoded_conv,
                    max_length=30, # maximum number of tokens
                    num_beams=10, # options kept active at the same time
                    num_return_sequences=10, # how many of the candidates to return, has to be <= num_beams
                    no_repeat_ngram_size=2, # this penalized probability of sequences with internal repetition
                    early_stopping=True,
                    #top_k= 50, 
                    #top_p = 0.6,
                    temperature= 2.0,
                    do_sample=True)

                for i, response in enumerate(responses):
                    responses_decoded1.append(self.p.tokenizer.decode(response, skip_special_tokens=True))
                    tempSentiment1.append(self.p.SIA.polarity_scores(responses_decoded1[i])['compound'])


                ID = min(range(len(tempSentiment1)), key=lambda i: abs(tempSentiment1[i]-self.preferedSentiment)) 

                tempAgent1.append(responses_decoded1[ID])
                tsentiment1.append(tempSentiment1[ID])

                tsentimentRange.append([min(tempSentiment1), max(tempSentiment1)])

                entireConv = entireConv + "    " + responses_decoded1[ID]

                #if (abs(mean(tsentiment1[j-(partner.p.disagreement_turns+1):j+1])-partner.preferedSentiment) > partner.p.dropout):
                #    if (j > (partner.p.min_conversation_length-2)):
                #        tempAgent2.append([])
                #        tsentiment2.append([])
                #        break
                

            
            #Turn for chatbot 2
            if j > 1:
                    entireConv = entireConv.split("    ", 1)[1]
            
            encoded_conv2 = partner.p.tokenizer(entireConv, return_tensors='pt').input_ids
            responses2 = partner.chatbot.generate(
                    encoded_conv2,
                    max_length=30, # maximum number of tokens
                    num_beams=10, # options kept active at the same time
                    num_return_sequences=10, # how many of the candidates to return, has to be <= num_beams
                    no_repeat_ngram_size=2, # this penalized probability of sequences with internal repetition
                    early_stopping=True,
                    #top_k= 50, 
                    #top_p = 0.6,
                    temperature= 2.0,
                    do_sample=True)
            
            for i, response in enumerate(responses2):
                    responses_decoded2.append(partner.p.tokenizer.decode(response, skip_special_tokens=True))
                    tempSentiment2.append(partner.p.SIA.polarity_scores(responses_decoded2[i])['compound'])

            partnerSent = partner.preferedSentiment
            
            ID = min(range(len(tempSentiment2)), key=lambda i: abs(tempSentiment2[i]-partnerSent)) 
            tempAgent2.append(responses_decoded2[ID])
            tsentiment2.append(tempSentiment2[ID])

            entireConv = entireConv + "    " + responses_decoded2[ID]
            

            
            
            #Determine if either chatbot will break out of the conversation 
            #(i.e. if difference in sentiment between prefered sentiment and sentiment of the other chatbot is greater than their tolerance)
            #if (abs(mean(tsentiment2[j-(self.p.disagreement_turns+1):j+1])-self.preferedSentiment) > self.p.dropout):
            #    if (j > (self.p.min_conversation_length-2)):
            #        break


                

            
            j += 1 #Add one to count
            

        self.agent1 = tempAgent1
        self.agent2 = tempAgent2
        
        self.conversation_starter = conversation_starter
        self.my_id = self.id
        self.partner_id = partner.id
        self.sentiment1 = tsentiment1
        self.sentiment2 = tsentiment2

        self.preferedSentiment1 = self.preferedSentiment
        self.preferedSentiment2 = partnerSent
        self.sentimentRange = tsentimentRange
        self.t = 1
        
        #Update prefered sentiment
        #if (j != (self.p.min_conversation_length-1)):
        #    self.preferedSentiment = (self.preferedSentiment+((mean(tsentiment2)-self.preferedSentiment)*self.p.learningRate*(j/self.p.conversation_length)))
        #    partner.preferedSentiment = (partner.preferedSentiment+((mean(tsentiment1)-partner.preferedSentiment)*partner.p.learningRate*(j/self.p.conversation_length)))

        self.preferedSentiment = (self.preferedSentiment+((mean(tsentiment2)-self.preferedSentiment)*self.p.learningRate))
        partner.preferedSentiment = (partner.preferedSentiment+((mean(tsentiment1)-partner.preferedSentiment)*partner.p.learningRate))

        self.environmentKnowledge.extend(tsentiment1)
        partner.environmentKnowledge.extend(tsentiment2)

        self.weighted_average = np.average(self.environmentKnowledge[-10:], weights=list(range(1,len(self.environmentKnowledge[-10:])+1)))
        partner.weighted_average = np.average(partner.environmentKnowledge[-10:], weights=list(range(1,len(partner.environmentKnowledge[-10:])+1)))


In [17]:
class ChatbotModel(ap.Model):

    """ """

    def setup(self):

        self.add_agents(self.p.agents, ChatbotAgent)

    def step(self):
        
        self.agents.chatbot_conversation()

    def update(self):


        self.record('Agent1', self.agents.agent1)
        self.record('Agent2', self.agents.agent2)

        self.record('Conversation_starter', self.agents.conversation_starter)

        self.record('id', self.agents.my_id)
        self.record('partner id', self.agents.partner_id)

        self.record('Sentiment1', self.agents.sentiment1)
        self.record('Sentiment2', self.agents.sentiment2)

        self.record('preferedSentiment1', self.agents.preferedSentiment1)
        self.record('preferedSentiment2', self.agents.preferedSentiment2)

        self.record('sentimentRange', self.agents.sentimentRange)
        self.record('envKnowledge', self.agents.environmentKnowledge)

    def end(self):
        pass


In [109]:
conversation_prompts = ("Let's talk about censorship and freedom of speech. What do you think about it?",
"Let's talk about climate change. What do you think about it?",
"Let's talk about death penalty. What do you think about it?",
"Let's talk about abortion. What do you think about it?",
"Let's talk about social security. What do you think about it?",
"Let's talk about health insurance. What do you think about it?",
"Let's talk about women's rights. What do you think about it?",
"Let's talk about religious freedom. What do you think about it?",
"Let's talk about minimum wage. What do you think about it?",
"Let's talk about atheism. What do you think about it?",
"Let's talk about reparations. What do you think about it?",
"Let's talk about hacking. What do you think about it?",
"Let's talk about labor unions. What do you think about it?",
"Let's talk about extremism. What do you think about it?",
"Let's talk about electoral college. What do you think about it?",
"Let's talk about vaccines. What do you think about it?",
"Let's talk about outsourcing. What do you think about it?",
"Let's talk about gun control. What do you think about it?",
"Let's talk about foreign aid. What do you think about it?",
"Let's talk about nuclear energy. What do you think about it?",
"Let's talk about police brutality. What do you think about it?")


parameters = {
    'agents': 50,
    'steps': 25,
    'conversation_length': 1, #.
    'conversation_starter': conversation_prompts, 
    'chatbot':my_bot1,
    'tokenizer':tokenizer,
    'SIA':SIA,
    'learningRate': 1,
    #'dropout': 2, #2 means it has no efffect
    #'min_conversation_length': 2, #OBS, if this is higher than conversation_length, there will be no learning and no dropout
    #'disagreement_turns': 2, #how many turns to take into account when figuring out if they agree or not.
    'beta': 0
}

In [110]:
model = ChatbotModel(parameters)
results = model.run()

Completed: 25 steps
Run time: 13:27:38.524306
Simulation finished


In [111]:
#Convert data into long format

#we can throw away the row at t=0, as this does not contain any information that we need and the agents does not have any interactions at this step
dataTemp = pd.DataFrame(results.variables)
dataTemp = dataTemp.drop([0])


#now create a number of lists for each agent
agent1 = []
agent2 = []
sentiment1 = []
sentiment2 = []
Id = []
partnerID = []
conversationStarter = []
turn = []
preferedSentiment1 = []
preferedSentiment2 = []
sentimentRange = []
step = []


#not optimal code, but works for now - create lists of correct length for each chatbot
for i in range(parameters['steps']):
    for j in range(parameters['agents']):
        for k in range(len(dataTemp['Agent1'][i+1][j])):
            #print(i+1,j,k)
            agent1.append(dataTemp['Agent1'][i+1][j][k])
            agent2.append(dataTemp['Agent2'][i+1][j][k])
            sentiment1.append(dataTemp['Sentiment1'][i+1][j][k])
            sentiment2.append(dataTemp['Sentiment2'][i+1][j][k])
            preferedSentiment1.append(dataTemp['preferedSentiment1'][i+1][j])
            preferedSentiment2.append(dataTemp['preferedSentiment2'][i+1][j])
            Id.append(dataTemp['id'][i+1][j])
            partnerID.append(dataTemp['partner id'][i+1][j])
            conversationStarter.append(dataTemp['Conversation_starter'][i+1][j])
            turn.append(k)
            sentimentRange.append(dataTemp['sentimentRange'][i+1][j][k])
            step.append(i+1)

In [112]:
#Create a new dataframe from the list
df = pd.DataFrame(
    {'Turn': turn,
     'Step': step,
     'Id': Id,
     'Partner_ID': partnerID,
     'Conversation_starter': conversationStarter,
     'Agent1': agent1,
     'Agent2': agent2,
     'Sentiment1': sentiment1,
     'Sentiment2': sentiment2,
     'preferedSentiment1':preferedSentiment1,
     'preferedSentiment2':preferedSentiment2,
     'sentimentRange': sentimentRange
    })

In [31]:
#View DF
df

,Turn,Step,Id,Partner_ID,Conversation_starter,Agent1,Agent2,Sentiment1,Sentiment2,preferedSentiment1,preferedSentiment2,sentimentRange
0,0,1,1,43,Let's talk about death penalty. What do you th...,I really like it! It forces a person to give ...,I know! I love how it lets people know there'...,-0.1025,0.0000,-0.006593,-0.185014,"[-0.9538, -0.1025]"
1,0,1,2,3,Let's talk about hacking. What do you think ab...,"I think that's a pretty good idea too, as lon...",I know my boss would have been furious. I ha...,0.8225,-0.7096,0.965193,-0.710338,"[-0.9136, 0.8225]"
2,0,1,3,11,Let's talk about foreign aid. What do you thin...,"i think that it is a wonderful thing, but i a...",I have no clue. It's so crazy that parents ca...,0.3291,-0.6504,-0.710338,-0.600403,"[0.3291, 0.9009]"
3,0,1,4,24,Let's talk about climate change. What do you t...,I think climate change is an issue but people...,I agree but it's a difficult topic to get int...,0.2960,-0.6486,0.204260,-0.789221,"[-0.8858, 0.7845]"
4,0,1,5,27,Let's talk about police brutality. What do you...,"I think it was pretty brutal, it seems like s...",I don't think I can stand people like that. I...,0.2023,0.2023,0.348942,0.216444,"[-0.9418, 0.2023]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1245,0,25,46,21,Let's talk about labor unions. What do you thi...,I think they're a bit of a joke. Some of them...,I know! I think it's really sad that people ...,-0.5092,-0.3415,-0.297214,-0.257978,"[-0.836, 0.7096]"
1246,0,25,47,15,Let's talk about health insurance. What do you...,"I do have insurance, which covers health insu...","Oh, that sounds like good insurance for your ...",0.0000,-0.0772,-0.050319,-0.248503,"[-0.631, 0.7096]"
1247,0,25,48,44,Let's talk about death penalty. What do you th...,I think it's horrible that some countries all...,"Oh yeah, and I bet a lot of those are false r...",-0.4588,0.0000,-0.304769,-0.046824,"[-0.7906, 0.6908]"
1248,0,25,49,4,Let's talk about electoral college. What do yo...,Electronic engineering is what I wanted to st...,"I know, it's got to involve a lot of differen...",0.0000,0.0772,-0.232249,0.204260,"[-0.5106, 0.8246]"


In [113]:
import time
now = time.strftime('%d-%m-%Y %H-%M-%S')
df.to_csv("experiment0_beta0_LR1_{}.csv".format(now))

In [36]:
results.variables

,Agent1,Agent2,Conversation_starter,id,partner id,Sentiment1,Sentiment2,preferedSentiment1,preferedSentiment2,sentimentRange,envKnowledge
t,,,,,,,,,,,
0,"[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[], []]","[[0, 0.6329, 0.5446], [0, -0.3, -0.2228]]"
1,"[[ I love it, it's so fun! It's a good time fo...",[[ I am looking forward to it so much! She ca...,[Let's talk about hacking. What do you think a...,"[1, 2]","[2, 1]","[[0.6329], [-0.2228]]","[[-0.3], [0.5446]]","[0.7842532259257606, -0.17151710334648174]","[-0.17151710334648174, 0.7842532259257606]","[[[-0.9480999999999999, 0.6329]], [[-1.0081, 0...","[[0, 0.6329, 0.5446], [0, -0.3, -0.2228]]"
